##PERGUNTAS QUE O MVP DEVE RESPONDER

###Padrões gerais de gasto
- Qual é o valor total gasto pela CEAP no período analisado?
- Qual é o gasto médio mensal por parlamentar?
- Como os gastos se distribuem ao longo dos meses do ano?
###Análise por parlamentar
- Quais parlamentares apresentam os maiores volumes de despesas?
- Existe grande variação de gastos entre parlamentares?
###Análise por tipo de despesa
- Quais tipos de despesa concentram a maior parte dos gastos da CEAP?
-- Ex: passagens aéreas, combustíveis, divulgação, hospedagem
###Análise por partido político
- Qual é o gasto médio por partido político?
- A composição de gastos varia entre partidos?
###Análise geográfica (UF)
- Como os gastos se distribuem entre as UFs?
###Análise temporal
- Há meses com picos atípicos de despesas?
- Determinados tipos de despesa concentram-se em períodos específicos do ano?


In [0]:
%sql
USE CATALOG mvp_ed_ceap;
USE SCHEMA layer_gold;

###Padrões gerais de gasto

In [0]:
%sql
-- Qual é o valor total gasto pela CEAP no período analisado?
SELECT
  SUM(valor_documento) AS gasto_total_ceap
FROM fato_despesa_ceap;


gasto_total_ceap
254190322.71


In [0]:
%sql
-- Qual é o gasto médio mensal por parlamentar?
SELECT
  p.nome_parlamentar,
  AVG(f.valor_documento) AS gasto_medio
FROM fato_despesa_ceap f
JOIN dim_parlamentar p
  ON f.id_parlamentar = p.id_parlamentar
GROUP BY p.nome_parlamentar
ORDER BY gasto_medio DESC
Limit(10); -- Adicionado limite para reduzir o output no Git

nome_parlamentar,gasto_medio
Glaustin da Fokus,14378.366250
PP,10363.636364
Gabriel Mota,9440.131667
Professor Alcides,7937.204194
Fábio Macedo,6330.956154
Silvio Antonio,6215.728261
Hildo do Candango,6148.172917
Alberto Fraga,5684.885854
Luciano Galego,5236.788966
Eunício Oliveira,4705.326190


In [0]:
%sql
--Como os gastos se distribuem ao longo dos meses do ano?
SELECT
  t.ano,
  t.mes,
  SUM(f.valor_documento) AS total_mes
FROM fato_despesa_ceap f
JOIN dim_tempo t
  ON f.id_tempo = t.id_tempo
GROUP BY t.ano, t.mes
ORDER BY t.ano, t.mes;


ano,mes,total_mes
2002,6,7900.00
2023,11,484.90
2023,12,232118.66
2024,1,12778217.68
2024,2,19311080.59
2024,3,21159356.29
2024,4,22387249.48
2024,5,21523297.59
2024,6,20987376.04
2024,7,19700276.64


---
###Análise por parlamentar

In [0]:
%sql
--Quais parlamentares apresentam os maiores volumes de despesas?
SELECT
  p.nome_parlamentar,
  SUM(f.valor_documento) AS total_gasto
FROM fato_despesa_ceap f
JOIN dim_parlamentar p
  ON f.id_parlamentar = p.id_parlamentar
GROUP BY p.nome_parlamentar
ORDER BY total_gasto DESC
LIMIT 10;


nome_parlamentar,total_gasto
Pompeo de Mattos,680276.40
Zé Haroldo Cathedral,633876.34
Lucas Ramos,626122.68
Gabriel Mota,623048.69
Átila Lins,622121.91
André Ferreira,621197.30
Defensor Stélio Dener,619937.12
Albuquerque,618342.81
Helena Lima,617655.57
Renilce Nicodemos,615122.95


In [0]:
%sql
--Existe grande variação de gastos entre parlamentares?
SELECT
  MIN(total_gasto) AS menor_gasto,
  MAX(total_gasto) AS maior_gasto,
  AVG(total_gasto) AS gasto_medio
FROM (
  SELECT
    id_parlamentar,
    SUM(valor_documento) AS total_gasto
  FROM fato_despesa_ceap
  GROUP BY id_parlamentar
);


menor_gasto,maior_gasto,gasto_medio
-6056.39,680276.40,427210.626403


---
###Análise por tipo de despesa

In [0]:
%sql
--Quais tipos de despesa concentram a maior parte dos gastos?
SELECT
  d.descricao_tipo_despesa,
  SUM(f.valor_documento) AS total_gasto
FROM fato_despesa_ceap f
JOIN dim_tipo_despesa d
  ON f.id_tipo_despesa = d.id_tipo_despesa
GROUP BY d.descricao_tipo_despesa
ORDER BY total_gasto DESC;


descricao_tipo_despesa,total_gasto
DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,100462740.70
COMBUSTÍVEIS E LUBRIFICANTES.,89823955.44
LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,42790994.12
PASSAGEM AÉREA - SIGEPA,40135176.19
MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR,32763711.68
"HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.",3565187.35
TELEFONIA,3144508.10
LOCAÇÃO OU FRETAMENTO DE AERONAVES,2154396.60
SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPECIALIZADA.,1593439.59
FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,1302040.24


---
###Análise por partido político

In [0]:
%sql
--Qual é o gasto médio por partido político?
SELECT
  p.sigla_partido,
  AVG(f.valor_documento) AS gasto_medio
FROM fato_despesa_ceap f
JOIN dim_partido p
  ON f.id_partido = p.id_partido
GROUP BY p.sigla_partido
ORDER BY gasto_medio DESC;


sigla_partido,gasto_medio
PV,1472.980291
PODE,1462.307255
PCDOB,1421.617017
SOLIDARIEDADE,1421.257145
REPUBLICANOS,1274.165899
MDB,1268.163539
PDT,1258.041026
UNIÃO,1252.724467
PSD,1203.860423
PP,1190.490642


In [0]:
%sql
--A composição de gastos varia entre partidos?
SELECT
  p.sigla_partido,
  d.descricao_tipo_despesa,
  SUM(f.valor_documento) AS total_gasto
FROM fato_despesa_ceap f
JOIN dim_partido p
  ON f.id_partido = p.id_partido
JOIN dim_tipo_despesa d
  ON f.id_tipo_despesa = d.id_tipo_despesa
GROUP BY p.sigla_partido, d.descricao_tipo_despesa
ORDER BY p.sigla_partido, total_gasto DESC
Limit(30); -- Adicionado limite para reduzir o output no Git


sigla_partido,descricao_tipo_despesa,total_gasto
AVANTE,COMBUSTÍVEIS E LUBRIFICANTES.,1724292.00
AVANTE,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,1103966.31
AVANTE,LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,931458.79
AVANTE,PASSAGEM AÉREA - SIGEPA,642068.35
AVANTE,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR,540898.31
AVANTE,"HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.",67208.58
AVANTE,TELEFONIA,59695.87
AVANTE,PASSAGEM AÉREA - RPA,23764.64
AVANTE,"PASSAGENS TERRESTRES, MARÍTIMAS OU FLUVIAIS",17759.75
AVANTE,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",16960.92


---
###Análise geográfica (UF)

In [0]:
%sql
--Como os gastos se distribuem entre as UFs?
SELECT
  u.sigla_uf,
  SUM(f.valor_documento) AS total_gasto
FROM fato_despesa_ceap f
JOIN dim_uf u
  ON f.id_uf = u.id_uf
GROUP BY u.sigla_uf
ORDER BY total_gasto DESC;


sigla_uf,total_gasto
SP,30635685.77
MG,25792197.78
RJ,20557252.54
BA,19784362.86
RS,15694851.89
PR,15023813.13
PE,13472003.04
CE,11818054.44
MA,9583239.75
PA,8291813.22


---
###Análise temporal

In [0]:
%sql
--Há meses com picos atípicos de despesas?
SELECT
  t.ano,
  t.mes,
  SUM(f.valor_documento) AS total_mes
FROM fato_despesa_ceap f
JOIN dim_tempo t
  ON f.id_tempo = t.id_tempo
GROUP BY t.ano, t.mes
ORDER BY total_mes DESC;



ano,mes,total_mes
2024,12,29317095.64
2024,11,23427224.87
2024,4,22387249.48
2024,5,21523297.59
2024,3,21159356.29
2024,6,20987376.04
2024,10,20245421.76
2024,7,19700276.64
2024,2,19311080.59
2024,8,18859586.04
